In [65]:
import pybaseball as pb
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import math
import statistics as stats

pb.cache.enable()

In [66]:
df_pitching = pd.read_csv("../lahman/core/pitching.csv")

In [67]:
df_pitching.columns()

TypeError: 'Index' object is not callable

In [78]:
df_pitching.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'W', 'L', 'G', 'GS',
       'CG', 'SHO', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp',
       'ERA', 'IBB', 'WP', 'HBP', 'BK', 'BFP', 'GF', 'R', 'SH', 'SF', 'GIDP'],
      dtype='object')

In [100]:
df = df_pitching
year_groups = df.groupby(['yearID']).apply(lambda x: [list(x['CG']), list(x['IPouts']), list(x['BAOpp']), list(x['BB']), list(x['SO']), list(x['ERA'])]).apply(pd.Series)
year_groups.columns = ['CG', 'IPOuts', 'BAOpp', 'BB', 'SO', 'ERA']
# cg_year_lists = (year_groups['CG']).apply(list)
# cg_year_lists = cg_year_lists.reset_index()

In [101]:
year_groups

,CG,IPOuts,BAOpp,BB,SO,ERA
yearID,,,,,,
1871,"[2, 30, 0, 22, 1, 0, 1, 19, 25, 28, 0, 1, 0, 2...","[78, 792, 3, 639, 27, 3, 39, 507, 666, 747, 3,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[11, 37, 0, 31, 3, 0, 3, 21, 40, 75, 2, 6, 3, ...","[1, 13, 0, 15, 0, 0, 1, 17, 15, 12, 0, 0, 0, 3...","[7.96, 4.5, 27.0, 4.35, 10.0, 0.0, 3.46, 5.17,..."
1872,"[14, 9, 1, 37, 5, 1, 53, 9, 2, 0, 9, 39, 47, 7...","[432, 237, 24, 1008, 177, 27, 1491, 330, 54, 1...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[12, 5, 0, 21, 3, 1, 31, 11, 4, 2, 9, 52, 26, ...","[5, 1, 0, 13, 5, 1, 45, 20, 0, 2, 3, 57, 44, 1...","[6.06, 6.38, 5.63, 4.53, 4.27, 7.0, 3.01, 1.8,..."
1873,"[1, 12, 51, 18, 42, 1, 5, 2, 1, 1, 0, 7, 1, 1,...","[48, 326, 1442, 495, 1146, 58, 253, 72, 54, 27...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2, 9, 41, 8, 33, 2, 10, 0, 1, 0, 1, 10, 1, 6,...","[0, 3, 16, 7, 34, 0, 14, 1, 0, 0, 0, 5, 1, 1, ...","[4.5, 4.14, 4.08, 2.95, 2.8, 5.59, 1.81, 5.25,..."
1874,"[4, 55, 25, 1, 52, 1, 31, 0, 0, 17, 62, 55, 0,...","[117, 1491, 720, 33, 1449, 27, 967, 21, 24, 53...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1, 8, 27, 2, 18, 3, 13, 0, 0, 12, 41, 32, 0, ...","[0, 42, 8, 0, 61, 0, 25, 0, 0, 12, 101, 37, 0,...","[1.62, 2.03, 3.71, 4.91, 1.96, 4.0, 2.32, 15.4..."
1875,"[0, 14, 4, 12, 37, 7, 0, 57, 18, 9, 46, 24, 0,...","[7, 378, 108, 387, 1056, 198, 5, 1607, 641, 36...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0, 5, 3, 2, 7, 7, 0, 17, 11, 5, 4, 12, 0, 0, ...","[0, 6, 3, 14, 70, 9, 0, 60, 9, 7, 82, 23, 0, 0...","[7.71, 2.71, 2.5, 3.07, 1.41, 1.5, 0.0, 2.13, ..."
...,...,...,...,...,...,...
2017,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[131, 15, 367, 123, 183, 30, 24, 72, 25, 22, 2...","[0.242, 0.211, 0.264, 0.262, 0.166, 0.194, 0.1...","[14, 8, 51, 10, 17, 6, 2, 12, 6, 4, 28, 21, 2,...","[37, 10, 108, 37, 63, 9, 5, 12, 10, 5, 59, 92,...","[3.3, 3.6, 5.52, 3.51, 1.62, 3.6, 1.13, 7.13, ..."
2018,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[97, 3, 23, 103, 102, 90, 198, 24, 201, 21, 44...","[0.246, 0.25, 0.267, 0.321, 0.214, 0.223, 0.23...","[15, 3, 4, 12, 23, 6, 27, 4, 33, 3, 4, 15, 22,...","[37, 0, 4, 32, 30, 21, 56, 3, 80, 7, 14, 23, 5...","[6.12, 0.0, 7.04, 7.34, 3.44, 2.4, 3.68, 12.38..."
2019,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[39, 26, 65, 8, 42, 3, 93, 76, 3, 220, 12, 35,...","[0.196, 0.138, 0.2, 0.333, 0.259, 0.0, 0.183, ...","[3, 3, 10, 3, 10, 2, 14, 11, 0, 18, 3, 4, 2, 2...","[9, 13, 18, 5, 9, 2, 51, 23, 1, 41, 6, 7, 0, 5...","[4.15, 1.04, 2.91, 16.88, 5.14, 9.0, 3.77, 8.5..."


In [96]:
cg_year_lists

,yearID,CG
0,1871,"[2, 30, 0, 22, 1, 0, 1, 19, 25, 28, 0, 1, 0, 2..."
1,1872,"[14, 9, 1, 37, 5, 1, 53, 9, 2, 0, 9, 39, 47, 7..."
2,1873,"[1, 12, 51, 18, 42, 1, 5, 2, 1, 1, 0, 7, 1, 1,..."
3,1874,"[4, 55, 25, 1, 52, 1, 31, 0, 0, 17, 62, 55, 0,..."
4,1875,"[0, 14, 4, 12, 37, 7, 0, 57, 18, 9, 46, 24, 0,..."
...,...,...
146,2017,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
147,2018,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
148,2019,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
149,2020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [74]:
year = 1871
season_list = []
counter = 0
for year in range(1871, 2021):
    season = cg_year_lists['yearID']
    season = {}
    season['year'] = cg_year_lists['yearID'][counter]
    season['team'] = cg_year_lists['teamID'][counter]
    season['CG_max'] = max(cg_year_lists['CG'][counter])
    season['CG_min'] = min(cg_year_lists['CG'][counter])
    season['CG_mean'] = stats.mean(cg_year_lists['CG'][counter])
    season['CG_median'] = stats.median(cg_year_lists['CG'][counter])
    season['CG_harmean'] = stats.harmonic_mean(cg_year_lists['CG'][counter])
    # season['CG_normaldist'] = stats.NormalDist(cg_year_lists['CG'][counter])
    # season['CG_stdev'] = stats.stdev(cg_year_lists['CG'][counter])
    season['CG_var'] = stats.variance(cg_year_lists['CG'][counter])
    season_list.append(season)
    counter += 1
    

In [75]:
season_list[0]

{'year': 1871,
 'CG_max': 31,
 'CG_min': 0,
 'CG_mean': 12.157894736842104,
 'CG_median': 2,
 'CG_harmean': 0,
 'CG_var': 161.69590643274853}

In [32]:
years = []
year_counter = 1871
for season in range(1871, 2021):
    year_counter = {}
    while df_pitching['yearID'] == year_counter:
        if df_pitching['CG'] >= 1:
            year_counter['CG_max'] = df_pitching['CG']


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [26]:
years.values()

dict_values([2020])